# 

# Here onwards i am build a chat appliation using RAG Framework

* Install the required packages

In [13]:
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet pinecone
%pip install --upgrade --quiet langchain-google-genai pillow
%pip install --upgrade --quiet langchain-pinecone
%pip install --upgrade --quiet python-dotenv pypdf
%pip install --upgrade --quiet sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


* Global imports

In [56]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeClient
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

* env-variables loading

In [26]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_HOST = os.getenv("PINECONE_API_HOST")
PINECONE_API_INDEX = os.getenv("PINECONE_API_INDEX")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

* Load the data/Extract the data from .pdf

In [27]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents = loader.load()
    return documents

In [28]:
extracted_data = load_pdf("../data/")

In [ ]:
extracted_data

* Data -> text_chunks

In [19]:
def text_split(extracted_data):
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_spliter.split_documents(extracted_data)
    return text_chunks

In [20]:
text_chunks = text_split(extracted_data)

In [21]:
print(f'length of total chunks is {len(text_chunks)}')

length of total chunks is 7020


* Download the embedding model from huggingface

In [30]:
download_hugging_face_embedding = lambda: HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [31]:
embedding = download_hugging_face_embedding()

In [32]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [34]:
vectors = embedding.embed_query("Hello, how are you?")

In [35]:
len(vectors)

384

* Pinecode initializing

In [36]:
pc = PineconeClient(api_key=PINECONE_API_KEY)

In [37]:
index_name = PINECONE_API_INDEX

In [38]:
dosearch = PineconeVectorStore.from_texts(
    [
        tc.page_content for tc in text_chunks
    ],
    embedding=embedding,
    index_name=index_name,
)

In [39]:
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

In [40]:
docsearch

* testing

In [41]:
query = "what is acne?"
docs = docsearch.similarity_search(query, k=4)
print(docs)

[Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'), Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'), Document(page_content='ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moderately severe acne.Isotretinoin (Accutane) is prescribed only for verysevere, disfiguring acne.\nAcne is a skin condition that occurs

* Prompt Templating

In [42]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [43]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

* Llm setup

In [44]:
def get_llm():
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        google_api_key=GOOGLE_API_KEY,
        temperature=1,
        max_output_tokens=512,
        min_output_tokens=12,
    )
    return llm


In [45]:
llm = get_llm()

In [46]:
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=1.0, max_output_tokens=512, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x38c5c5590>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x31f216090>, default_metadata=())

* Llm chains

In [47]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 4}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [57]:
res = qa.invoke({"query": "what are allergies"})

In [58]:
print(res)

{'query': 'what are allergies', 'result': 'Allergies are reactions by the immune system to substances that are normally harmless, such as dust, pollen, or pet dander. These substances are called allergens.', 'source_documents': [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommo

In [59]:
print(res["result"])

Allergies are reactions by the immune system to substances that are normally harmless, such as dust, pollen, or pet dander. These substances are called allergens.


In [60]:
"done🤘🏻"

'done🤘🏻'